In [10]:
from pydub import AudioSegment

video = AudioSegment.from_file("../samples/videoplayback.mp4", format="mp4")

In [11]:
audio = video.set_channels(1).set_frame_rate(16000).set_sample_width(2)

In [12]:
audio.export("../samples/out/videoplayback.wav", format="wav")

<_io.BufferedRandom name='../samples/out/videoplayback.wav'>

In [2]:
import whisper
model = whisper.load_model("base")

In [4]:
result = model.transcribe("../samples/out/videoplayback.wav")

/opt/miniconda3/envs/sportlight/lib/python3.9/site-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [6]:
result["text"]

" This is where you felt something's going to give. He's been itching to get on strike. First boundary of the final. Cuts and cuts hard. That's typical del Shun. Agent Taken. Good catch at first lip. Pacen bound in this time movement too for the you can't get the outside edge. And they have claimed up with Tarenga. Lovely shot. The Sangekar up cover drive. And so Sangekar has to go over the field if they're stopping them on the ground take the aerial route. Another nice drive down the ground. Savileg's four more back to back boundaries. As he gone. As he been bowled round his legs. I think so. The Indian celebrate. Come off some part of his body. Harbhajan takes the wicked. Let's hit in the gap and that would go all the way for four. Shot delivery pulled away in the gap. And then he's going to get the shot. And then he's going to get the shot. The shot delivery pulled away in the gap. Would get a boundary from Mahela. Does play that shot well. Now that's a. A classy shot from Mahela J.